Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    sets = pickle.load(f)
    save = sets['sanitized']
    train_dataset = save[2][1]
    train_labels = save[2][2]
    valid_dataset = save[1][1]
    valid_labels = save[1][2]
    test_dataset = save[0][1]
    test_labels = save[0][2]
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (195106, 28, 28) (195106,)
Validation set (9840, 28, 28) (9840,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195106, 784) (195106, 10)
Validation set (9840, 784) (9840, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [7]:
num_steps = 801

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.667938
Training accuracy: 10.2%
Validation accuracy: 14.5%
Test accuracy: 14.6%
Loss at step 100: 2.394966
Training accuracy: 71.1%
Validation accuracy: 70.5%
Test accuracy: 77.6%
Loss at step 200: 1.930415
Training accuracy: 74.0%
Validation accuracy: 72.7%
Test accuracy: 79.8%
Loss at step 300: 1.673183
Training accuracy: 75.7%
Validation accuracy: 73.5%
Test accuracy: 80.8%
Loss at step 400: 1.497941
Training accuracy: 76.5%
Validation accuracy: 73.8%
Test accuracy: 81.4%
Loss at step 500: 1.367055
Training accuracy: 77.1%
Validation accuracy: 74.0%
Test accuracy: 81.6%
Loss at step 600: 1.264567
Training accuracy: 77.7%
Validation accuracy: 74.2%
Test accuracy: 82.0%
Loss at step 700: 1.181595
Training accuracy: 78.3%
Validation accuracy: 74.5%
Test accuracy: 82.3%
Loss at step 800: 1.112657
Training accuracy: 78.8%
Validation accuracy: 74.7%
Test accuracy: 82.5%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.823532
Minibatch accuracy: 20.3%
Validation accuracy: 15.5%
Minibatch loss at step 500: 1.696258
Minibatch accuracy: 75.0%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.065162
Minibatch accuracy: 78.9%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 1.138724
Minibatch accuracy: 76.6%
Validation accuracy: 76.9%
Minibatch loss at step 2000: 1.065887
Minibatch accuracy: 75.8%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.814829
Minibatch accuracy: 82.0%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.870797
Minibatch accuracy: 83.6%
Validation accuracy: 78.5%
Test accuracy: 86.3%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
num_relus = 1024

def make_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def make_bias(shape, constant=0.1):
    return tf.Variable(tf.constant(constant, shape=shape))

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    
    step1_weights = make_weights([image_size * image_size, num_relus])
    step1_biases = make_bias([num_relus])
    
    step2_weights = make_weights([num_relus, num_labels])
    step2_biases = make_bias([num_labels])
    
    # Training computation.
    def run_on_data(data):
        step1_logits = tf.matmul(data, step1_weights) + step1_biases
        intermediate_step = tf.nn.relu(step1_logits)
        my_logits = tf.matmul(intermediate_step, step2_weights) + step2_biases
        return my_logits
    
    logits = run_on_data(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(run_on_data(tf_valid_dataset))
    test_prediction = tf.nn.softmax(run_on_data(tf_test_dataset))


In [16]:
from timeit import default_timer as timer

num_steps = 100000
num_prints = 100

iteration_log = []
def log_step(start_time, duration, progress, loss, batch_acc, valid_acc=False, test_acc=False):
    iteration_log.append([start_time, duration, progress, loss, batch_acc, valid_acc, test_acc])
    print('>>>>>', int(progress * 100), '% done')
    print("time to execute last %d minibatches: %f" % (int(num_steps / num_prints), duration))
    print()
    print("loss of last minibatch %f" % loss)
    print("accuracy on last minibatch: %.1f%%" % batch_acc)
    print()
    if (valid_acc):
        print("model's current accuracy against validation data : %.1f%%" % valid_acc)
    if (test_acc):
        print("model's current accuracy against test data       : %.1f%%" % test_acc)
    print()
    

with tf.Session(graph=graph) as session:
    initial_time = timer()
    start = timer()
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        
        # run
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        
        if (step % int(num_steps / num_prints) == 0):
            end = timer()
            log_step(start, end - start, step / num_steps, l,
              accuracy(predictions, batch_labels),
              accuracy(valid_prediction.eval(), valid_labels),
              accuracy(test_prediction.eval(), test_labels)
            )
            start = timer()
    print()
    print("Total time for %d iterations: %d" % (num_steps, timer() - initial_time))


Initialized
>>>>> 0 % done
time to execute last 1000 minibatches: 0.364086

loss of last minibatch 4.263150
accuracy on last minibatch: 15.6%

model's current accuracy against validation data : 37.1%
model's current accuracy against test data       : 42.0%

>>>>> 1 % done
time to execute last 1000 minibatches: 2.439389

loss of last minibatch 0.384885
accuracy on last minibatch: 86.7%

model's current accuracy against validation data : 85.9%
model's current accuracy against test data       : 92.8%

>>>>> 2 % done
time to execute last 1000 minibatches: 2.319521

loss of last minibatch 0.308145
accuracy on last minibatch: 90.6%

model's current accuracy against validation data : 87.2%
model's current accuracy against test data       : 93.4%

>>>>> 3 % done
time to execute last 1000 minibatches: 2.347231

loss of last minibatch 0.293584
accuracy on last minibatch: 91.4%

model's current accuracy against validation data : 87.8%
model's current accuracy against test data       : 94.1%

>>>>